# <center> Twitter API </center>

In [ ]:
import sys
import json
import pymongo
from datetime import datetime
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from pymongo import MongoClient

## insert your mongoDB access key
client = MongoClient('your mongoDB access key here')

db = client.twitter219debate
scPrimary = db.scPrimary
class StdOutListener(StreamListener):
    def on_data(self, data):
        full_tweet = json.loads(data)
        ##this makes sure that you won't get clipped tweets
        if 'extended_tweet' in full_tweet:
            tweet_text = full_tweet.get('full_text')
        else:
            tweet_text = full_tweet.get('text')
        tweet_time = full_tweet.get('created_at')
        tweet_lang = full_tweet.get('lang')
        if tweet_lang != None and tweet_lang == 'en' and tweet_text != None and 'RT @' not in tweet_text:
            ##this is only taking the text and the time stamp, which is making the DB very space effecient
            tweetObject = {
            "text": tweet_text,
            "time": tweet_time,
            }
            scPrimary.insert_one(tweetObject)
            return True
    def on_error(self, status):
        print (status)
        
keyFile = open('--your twitter dev API key here--','r')
CONSUMER_KEY = keyFile.readline().rstrip()
CONSUMER_SECRET = keyFile.readline().rstrip()
ACCESS_TOKEN_KEY = keyFile.readline().rstrip()
ACCESS_TOKEN_SECRET = keyFile.readline().rstrip()

auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN_KEY, ACCESS_TOKEN_SECRET)

while True:
    try:
        stream = Stream(auth, StdOutListener())
        stream.filter(track=[]) ##insert your filters here
    except KeyboardInterrupt:
        s.close()
        stream.disconnect()
        break